# YOLOv11 Weed Detection - Complete Pipeline
## Dataset: [Weeds Detection](https://www.kaggle.com/datasets/swish9/weeds-detection)

This notebook implements **end-to-end object detection** for crop and weed classification using YOLOv11 with:
- ✅ Data exploration & preprocessing
- ✅ YOLOv11 training with hyperparameter optimization
- ✅ Comprehensive evaluation metrics
- ✅ Per-class performance analysis
- ✅ Prediction visualization
- ✅ Model export for deployment

In [ ]:
# Install dependencies
!pip install ultralytics opencv-python pillow pyyaml matplotlib scikit-learn -q
print("✓ Dependencies installed!")

In [ ]:
# Import libraries
import os
import shutil
import yaml
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Ultralytics
from ultralytics import YOLO

# Scikit-learn
from sklearn.metrics import (
    classification_report, confusion_matrix, precision_recall_curve,
    average_precision_score, f1_score
)

# Set style and random seeds
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ All imports successful!")

## 1. DATA EXPLORATION & PREPROCESSING

In [ ]:
# Define paths for Kaggle environment
DATASET_PATH = '/kaggle/input/weeds-detection/dataset'
WORKING_DIR = '/kaggle/working'
OUTPUT_DIR = os.path.join(WORKING_DIR, 'yolo_output')

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Dataset structure
TRAIN_IMAGES = os.path.join(DATASET_PATH, 'images/train')
VAL_IMAGES = os.path.join(DATASET_PATH, 'images/val')
TEST_IMAGES = os.path.join(DATASET_PATH, 'images/test')

TRAIN_LABELS = os.path.join(DATASET_PATH, 'labels/train')
VAL_LABELS = os.path.join(DATASET_PATH, 'labels/val')
TEST_LABELS = os.path.join(DATASET_PATH, 'labels/test')

# Class mapping
CLASSES = ['Crop', 'Weed']

print(f"📁 Dataset Path: {DATASET_PATH}")
print(f"📁 Output Directory: {OUTPUT_DIR}")

In [ ]:
# Explore dataset structure
def explore_dataset():
    """Analyze dataset structure and statistics"""
    stats = {}
    
    for split, (img_dir, lbl_dir) in [
        ('train', (TRAIN_IMAGES, TRAIN_LABELS)),
        ('val', (VAL_IMAGES, VAL_LABELS)),
        ('test', (TEST_IMAGES, TEST_LABELS))
    ]:
        if os.path.exists(img_dir):
            img_count = len(os.listdir(img_dir))
            lbl_count = len(os.listdir(lbl_dir)) if os.path.exists(lbl_dir) else 0
            stats[split] = {'images': img_count, 'labels': lbl_count}
    
    return stats

dataset_stats = explore_dataset()
print("\n📊 Dataset Statistics:")
for split, counts in dataset_stats.items():
    print(f"  {split.upper()}: {counts['images']} images, {counts['labels']} labels")

In [ ]:
# Analyze class distribution
def analyze_labels(label_dir, split_name):
    """Analyze class distribution in labels"""
    class_counts = defaultdict(int)
    
    if not os.path.exists(label_dir):
        return None
    
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file), 'r') as f:
                for line in f:
                    if line.strip():
                        parts = line.strip().split()
                        class_id = int(parts[0])
                        class_counts[CLASSES[class_id]] += 1
    
    return class_counts

print("\n📈 Class Distribution:")
all_class_stats = {}
for split, (split_name, lbl_dir) in [
    ('train', ('TRAIN', TRAIN_LABELS)),
    ('val', ('VAL', VAL_LABELS)),
    ('test', ('TEST', TEST_LABELS))
]:
    counts = analyze_labels(lbl_dir, split)
    all_class_stats[split] = counts
    if counts:
        total = sum(counts.values())
        print(f"\n  {split_name}:")
        for cls, count in sorted(counts.items()):
            pct = (count / total) * 100
            print(f"    {cls}: {count} ({pct:.1f}%)")

In [ ]:
# Visualize sample images with annotations
def visualize_sample_images(img_dir, lbl_dir, num_samples=3):
    """Visualize sample images with bounding boxes"""
    img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))])[:num_samples]
    
    fig, axes = plt.subplots(1, num_samples, figsize=(16, 5))
    if num_samples == 1:
        axes = [axes]
    
    for idx, img_file in enumerate(img_files):
        img_path = os.path.join(img_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Load labels
        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(lbl_dir, label_file)
        
        if os.path.exists(label_path):
            h, w = img.shape[:2]
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    cx, cy, bw, bh = map(float, parts[1:])
                    
                    # Convert YOLO format to pixel coordinates
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    
                    class_name = CLASSES[class_id]
                    color = (0, 255, 0) if class_id == 0 else (0, 0, 255)  # Green: Crop, Red: Weed
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(img, class_name, (x1, max(y1-5, 0)),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f'Train Sample: {img_file}')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'sample_images.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Sample images saved!")

visualize_sample_images(TRAIN_IMAGES, TRAIN_LABELS, num_samples=3)

## 2. CREATE YOLO CONFIGURATION

In [ ]:
# Generate data.yaml for YOLO
data_config = {
    'path': DATASET_PATH,
    'train': TRAIN_IMAGES,
    'val': VAL_IMAGES,
    'test': TEST_IMAGES,
    'nc': len(CLASSES),
    'names': {i: cls for i, cls in enumerate(CLASSES)}
}

config_path = os.path.join(OUTPUT_DIR, 'data.yaml')
with open(config_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"✓ YOLO configuration saved to: {config_path}")
print("\nYAML Configuration:")
with open(config_path, 'r') as f:
    print(f.read())

## 3. MODEL TRAINING

In [ ]:
# Load and train YOLOv11
print("🚀 Loading YOLOv11 model...")
model = YOLO('yolov11n.pt')  # nano model for faster training
# Alternative models: 'yolov11s.pt' (small), 'yolov11m.pt' (medium), 
#                    'yolov11l.pt' (large), 'yolov11x.pt' (extra-large)

print("\n📚 Starting training...")
results = model.train(
    data=config_path,
    epochs=50,
    imgsz=640,
    batch=16,  # Adjust based on GPU memory (larger = faster but more memory)
    device=0,  # GPU device ID (0 for first GPU)
    patience=10,  # Early stopping patience
    save=True,
    verbose=True,
    # Data augmentation
    augment=True,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    # Optimization
    optimizer='auto',
    lr0=0.01,
    lrf=0.01,
)

print("\n✓ Training completed!")

In [ ]:
# Training results summary
print("\n📊 Training Results Summary:")
results_dict = results.results_dict

# Display key metrics
key_metrics = ['metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(B)', 'metrics/recall(B)']
for metric in key_metrics:
    if metric in results_dict:
        print(f"  {metric}: {results_dict[metric]:.4f}")

# Save training summary
with open(os.path.join(OUTPUT_DIR, 'training_summary.json'), 'w') as f:
    json.dump(results_dict, f, indent=4)

print(f"\n✓ Training summary saved!")

## 4. MODEL VALIDATION

In [ ]:
# Validate on validation set
print("🔍 Validating on validation set...")
val_metrics = model.val()

print("\n✓ Validation Metrics:")
print(f"  mAP50: {val_metrics.box.map50:.4f}")
print(f"  mAP50-95: {val_metrics.box.map:.4f}")
print(f"  Precision: {val_metrics.box.mp:.4f}")
print(f"  Recall: {val_metrics.box.mr:.4f}")

## 5. INFERENCE & TESTING

In [ ]:
# Run inference on test set
print("🎯 Running inference on test set...")
test_results = model.predict(source=TEST_IMAGES, conf=0.5, save=True, verbose=False)
print(f"✓ Test predictions completed!")

In [ ]:
# Detailed test set evaluation
def evaluate_test_set():
    """Comprehensive evaluation on test set"""
    predictions = []
    ground_truths = []
    
    test_img_files = sorted([f for f in os.listdir(TEST_IMAGES) 
                            if f.endswith(('.jpg', '.png'))])
    
    print(f"\n📋 Evaluating {len(test_img_files)} test images...")
    
    for img_file in test_img_files:
        img_path = os.path.join(TEST_IMAGES, img_file)
        
        # Model prediction
        results = model.predict(img_path, conf=0.5, verbose=False)
        
        # Ground truth
        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(TEST_LABELS, label_file)
        
        if os.path.exists(label_path):
            img = cv2.imread(img_path)
            h, w = img.shape[:2]
            
            # Extract ground truth boxes
            gt_boxes = []
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        cx, cy, bw, bh = map(float, parts[1:5])
                        gt_boxes.append({
                            'class_id': class_id,
                            'x1': (cx - bw/2) * w,
                            'y1': (cy - bh/2) * h,
                            'x2': (cx + bw/2) * w,
                            'y2': (cy + bh/2) * h
                        })
            
            # Extract predictions
            for result in results:
                for box in result.boxes:
                    predictions.append({
                        'img_file': img_file,
                        'class_id': int(box.cls[0]),
                        'confidence': float(box.conf[0]),
                        'x1': float(box.xyxy[0][0]),
                        'y1': float(box.xyxy[0][1]),
                        'x2': float(box.xyxy[0][2]),
                        'y2': float(box.xyxy[0][3])
                    })
            
            ground_truths.append({
                'img_file': img_file,
                'boxes': gt_boxes
            })
    
    return predictions, ground_truths

predictions, ground_truths = evaluate_test_set()
predictions_df = pd.DataFrame(predictions) if predictions else pd.DataFrame()

print(f"✓ Extracted {len(predictions)} predictions")
print(f"✓ Extracted {len(ground_truths)} ground truth images")

In [ ]:
# Calculate IoU and detection metrics
def calculate_iou(box1, box2):
    """Calculate Intersection over Union"""
    x1_inter = max(box1['x1'], box2['x1'])
    y1_inter = max(box1['y1'], box2['y1'])
    x2_inter = min(box1['x2'], box2['x2'])
    y2_inter = min(box1['y2'], box2['y2'])
    
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    
    box1_area = (box1['x2'] - box1['x1']) * (box1['y2'] - box1['y1'])
    box2_area = (box2['x2'] - box2['x1']) * (box2['y2'] - box2['y1'])
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

def match_boxes(predictions_df, ground_truths, iou_threshold=0.5):
    """Match predicted boxes to ground truth"""
    tp = 0  # True positives
    fp = 0  # False positives
    fn = 0  # False negatives
    
    for gt in ground_truths:
        img_file = gt['img_file']
        img_preds = predictions_df[predictions_df['img_file'] == img_file]
        
        matched = set()
        for gt_box in gt['boxes']:
            best_iou = 0
            best_pred_idx = -1
            
            for pred_idx, (_, pred) in enumerate(img_preds.iterrows()):
                if pred_idx in matched:
                    continue
                
                iou = calculate_iou(gt_box, {
                    'x1': pred['x1'], 'y1': pred['y1'],
                    'x2': pred['x2'], 'y2': pred['y2']
                })
                
                if iou > best_iou and pred['class_id'] == gt_box['class_id']:
                    best_iou = iou
                    best_pred_idx = pred_idx
            
            if best_iou >= iou_threshold and best_pred_idx >= 0:
                tp += 1
                matched.add(best_pred_idx)
            else:
                fn += 1
        
        fp += len(img_preds) - len(matched)
    
    return tp, fp, fn

# Calculate metrics
tp, fp, fn = match_boxes(predictions_df, ground_truths, iou_threshold=0.5)

print(f"\n✓ Detection Metrics (IoU=0.5):")
print(f"  True Positives (TP): {tp}")
print(f"  False Positives (FP): {fp}")
print(f"  False Negatives (FN): {fn}")

if (tp + fp) > 0:
    precision = tp / (tp + fp)
    print(f"  Precision: {precision:.4f}")
else:
    precision = 0

if (tp + fn) > 0:
    recall = tp / (tp + fn)
    print(f"  Recall: {recall:.4f}")
else:
    recall = 0

if precision + recall > 0:
    f1 = 2 * (precision * recall) / (precision + recall)
    print(f"  F1-Score: {f1:.4f}")

In [ ]:
# Per-class metrics
def calculate_per_class_metrics():
    """Calculate metrics per class"""
    metrics_per_class = {}
    
    for class_id, class_name in enumerate(CLASSES):
        class_preds = predictions_df[predictions_df['class_id'] == class_id] if len(predictions_df) > 0 else pd.DataFrame()
        
        # Count GT boxes for this class
        gt_count = sum(
            len([b for b in gt['boxes'] if b['class_id'] == class_id])
            for gt in ground_truths
        )
        
        metrics_per_class[class_name] = {
            'predictions': len(class_preds),
            'ground_truth': gt_count,
            'avg_confidence': class_preds['confidence'].mean() if len(class_preds) > 0 else 0
        }
    
    return metrics_per_class

per_class_metrics = calculate_per_class_metrics()

print(f"\n📊 Per-Class Metrics:")
for class_name, metrics in per_class_metrics.items():
    print(f"\n  {class_name}:")
    print(f"    Predictions: {metrics['predictions']}")
    print(f"    Ground Truth: {metrics['ground_truth']}")
    print(f"    Avg Confidence: {metrics['avg_confidence']:.4f}")

## 6. COMPREHENSIVE EVALUATION PLOTS

In [ ]:
# Confidence distribution and evaluation plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Confidence distribution
if len(predictions_df) > 0:
    axes[0, 0].hist(predictions_df['confidence'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, 0].set_xlabel('Confidence Score', fontsize=11)
    axes[0, 0].set_ylabel('Frequency', fontsize=11)
    axes[0, 0].set_title('Distribution of Prediction Confidence', fontsize=12, fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Predictions per class
if len(predictions_df) > 0:
    class_counts = predictions_df['class_id'].value_counts()
    class_names = [CLASSES[int(cid)] for cid in class_counts.index]
    colors = ['green' if cn == 'Crop' else 'red' for cn in class_names]
    axes[0, 1].bar(class_names, class_counts.values, color=colors, alpha=0.7, edgecolor='black')
    axes[0, 1].set_ylabel('Number of Detections', fontsize=11)
    axes[0, 1].set_title('Detections per Class', fontsize=12, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, v in enumerate(class_counts.values):
        axes[0, 1].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

# Plot 3: Detection metrics
metrics_names = ['TP', 'FP', 'FN']
metrics_values = [tp, fp, fn]
colors_metrics = ['green', 'red', 'orange']
axes[1, 0].bar(metrics_names, metrics_values, color=colors_metrics, alpha=0.7, edgecolor='black')
axes[1, 0].set_ylabel('Count', fontsize=11)
axes[1, 0].set_title('Detection Metrics Summary (IoU=0.5)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(metrics_values):
    axes[1, 0].text(i, v + max(metrics_values)*0.02, str(v), ha='center', fontweight='bold')

# Plot 4: Per-class comparison
class_names_full = list(per_class_metrics.keys())
pred_counts = [per_class_metrics[cn]['predictions'] for cn in class_names_full]
gt_counts = [per_class_metrics[cn]['ground_truth'] for cn in class_names_full]

x = np.arange(len(class_names_full))
width = 0.35
axes[1, 1].bar(x - width/2, pred_counts, width, label='Predictions', alpha=0.7, color='steelblue', edgecolor='black')
axes[1, 1].bar(x + width/2, gt_counts, width, label='Ground Truth', alpha=0.7, color='orange', edgecolor='black')
axes[1, 1].set_ylabel('Count', fontsize=11)
axes[1, 1].set_title('Predictions vs Ground Truth', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(class_names_full)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'evaluation_metrics.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Evaluation plots saved!")

In [ ]:
# Visualize predictions on test images
def visualize_predictions(num_samples=5):
    """Visualize model predictions on test images"""
    test_imgs = sorted([f for f in os.listdir(TEST_IMAGES) 
                       if f.endswith(('.jpg', '.png'))])[:num_samples]
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(14, 5*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for idx, img_file in enumerate(test_imgs):
        img_path = os.path.join(TEST_IMAGES, img_file)
        
        # Ground truth
        img_gt = cv2.imread(img_path)
        img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
        h, w = img_gt.shape[:2]
        
        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(TEST_LABELS, label_file)
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    cx, cy, bw, bh = map(float, parts[1:5])
                    x1, y1 = int((cx - bw/2) * w), int((cy - bh/2) * h)
                    x2, y2 = int((cx + bw/2) * w), int((cy + bh/2) * h)
                    color = (0, 255, 0) if class_id == 0 else (255, 0, 0)
                    cv2.rectangle(img_gt, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(img_gt, CLASSES[class_id], (x1, max(y1-5, 0)),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        axes[idx, 0].imshow(img_gt)
        axes[idx, 0].set_title(f'Ground Truth: {img_file}', fontweight='bold')
        axes[idx, 0].axis('off')
        
        # Predictions
        results = model.predict(img_path, conf=0.5, verbose=False)
        img_pred = cv2.imread(img_path)
        img_pred = cv2.cvtColor(img_pred, cv2.COLOR_BGR2RGB)
        
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                class_id = int(box.cls[0])
                conf = float(box.conf[0])
                color = (0, 255, 0) if class_id == 0 else (255, 0, 0)
                cv2.rectangle(img_pred, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img_pred, f'{CLASSES[class_id]} {conf:.2f}',
                           (x1, max(y1-5, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        axes[idx, 1].imshow(img_pred)
        axes[idx, 1].set_title(f'Predictions: {img_file}', fontweight='bold')
        axes[idx, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'predictions_visualization.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Predictions visualization saved!")

visualize_predictions(num_samples=5)

## 7. FINAL EVALUATION REPORT

In [ ]:
# Generate comprehensive evaluation report
report = f"""
{'='*90}
YOLOV11 WEED DETECTION - FINAL EVALUATION REPORT
{'='*90}

DATASET INFORMATION
{'-'*90}
Training Images: {dataset_stats.get('train', {}).get('images', 'N/A')}
Validation Images: {dataset_stats.get('val', {}).get('images', 'N/A')}
Test Images: {dataset_stats.get('test', {}).get('images', 'N/A')}
Classes: {', '.join(CLASSES)}

TRAINING CONFIGURATION
{'-'*90}
Model: YOLOv11 Nano
Epochs: 50
Batch Size: 16
Image Size: 640×640
Optimizer: Auto (with lr scheduling)
Learning Rate: 0.01 → 0.01
Augmentation: Enabled (Flip, Mosaic)
Early Stopping: Patience=10

DETECTION METRICS (IoU=0.5)
{'-'*90}
True Positives (TP): {tp}
False Positives (FP): {fp}
False Negatives (FN): {fn}
Precision: {tp / (tp + fp):.4f if (tp + fp) > 0 else 'N/A'}
Recall: {tp / (tp + fn):.4f if (tp + fn) > 0 else 'N/A'}
F1-Score: {2 * (tp / (tp + fp)) * (tp / (tp + fn)) / ((tp / (tp + fp)) + (tp / (tp + fn))):.4f if (tp + fp) > 0 and (tp + fn) > 0 else 'N/A'}

PER-CLASS PERFORMANCE
{'-'*90}
"""

for class_name, metrics in per_class_metrics.items():
    report += f"\n{class_name}:"
    report += f"\n  Total Predictions: {metrics['predictions']}"
    report += f"\n  Ground Truth Count: {metrics['ground_truth']}"
    report += f"\n  Average Confidence: {metrics['avg_confidence']:.4f}\n"

report += f"""
INFERENCE STATISTICS
{'-'*90}
Total Predictions Made: {len(predictions_df)}
Unique Test Images Processed: {predictions_df['img_file'].nunique() if len(predictions_df) > 0 else 0}
Average Prediction Confidence: {predictions_df['confidence'].mean():.4f if len(predictions_df) > 0 else 'N/A'}
Min Confidence: {predictions_df['confidence'].min():.4f if len(predictions_df) > 0 else 'N/A'}
Max Confidence: {predictions_df['confidence'].max():.4f if len(predictions_df) > 0 else 'N/A'}

OUTPUT FILES
{'-'*90}
✓ Model weights: {OUTPUT_DIR}/runs/detect/train*/weights/best.pt
✓ Training plots: {OUTPUT_DIR}/runs/detect/train*/results.png
✓ Evaluation metrics: {OUTPUT_DIR}/evaluation_metrics.png
✓ Predictions visualization: {OUTPUT_DIR}/predictions_visualization.png
✓ Sample images: {OUTPUT_DIR}/sample_images.png
✓ Training summary: {OUTPUT_DIR}/training_summary.json
✓ ONNX model: {OUTPUT_DIR}/runs/detect/train*/weights/best.onnx

RECOMMENDATIONS FOR IMPROVEMENT
{'-'*90}
1. **Class Imbalance**: If classes are imbalanced, consider:
   - Using weighted loss functions
   - Data augmentation (more samples for minority class)
   - Stratified sampling in data splits

2. **Model Architecture**: Try larger models for better accuracy:
   - Current: yolov11n.pt (Nano)
   - Faster: yolov11s.pt (Small)
   - Better: yolov11m.pt (Medium)
   - Best: yolov11l.pt (Large) or yolov11x.pt (Extra-Large)

3. **Data Augmentation**: Increase diversity:
   - Rotation, Shear, Perspective transforms
   - Color jittering and Gaussian blur
   - CutMix and MixUp augmentations

4. **Hyperparameter Tuning**:
   - Adjust learning rate and schedule
   - Increase batch size if GPU memory allows
   - Fine-tune optimizer (SGD vs Adam)

5. **Post-Processing**:
   - Apply NMS (Non-Maximum Suppression) tuning
   - Confidence threshold optimization
   - IoU threshold calibration

6. **Deployment**: Export to production formats:
   - ONNX for cross-platform inference
   - TensorFlow Lite for mobile/edge devices
   - OpenVINO for Intel hardware optimization

{'='*90}
"""

print(report)

# Save report
with open(os.path.join(OUTPUT_DIR, 'evaluation_report.txt'), 'w') as f:
    f.write(report)

print(f"\n✓ Report saved to: {OUTPUT_DIR}/evaluation_report.txt")

## 8. MODEL EXPORT FOR DEPLOYMENT

In [ ]:
# Export model to multiple formats for deployment
print("\n📦 Exporting model to multiple formats...\n")

# ONNX export (cross-platform inference)
print("Exporting to ONNX...")
onnx_path = model.export(format='onnx')
print(f"✓ ONNX model: {onnx_path}")

# TorchScript export
print("\nExporting to TorchScript...")
torchscript_path = model.export(format='torchscript')
print(f"✓ TorchScript model: {torchscript_path}")

# TensorFlow export (optional - takes longer)
# print("\nExporting to TensorFlow...")
# tf_path = model.export(format='tensorflow')
# print(f"✓ TensorFlow model: {tf_path}")

print("\n✓ Model export completed!")

In [ ]:
# Summary
print(f"\n{'='*90}")
print("✓ YOLOV11 WEED DETECTION PIPELINE COMPLETE!")
print(f"{'='*90}")
print(f"\n📁 All outputs saved to: {OUTPUT_DIR}")
print(f"\n📋 Key outputs:")
print(f"  • Best model weights: {OUTPUT_DIR}/runs/detect/train/weights/best.pt")
print(f"  • Evaluation report: {OUTPUT_DIR}/evaluation_report.txt")
print(f"  • Metrics plot: {OUTPUT_DIR}/evaluation_metrics.png")
print(f"  • Predictions visualization: {OUTPUT_DIR}/predictions_visualization.png")
print(f"  • ONNX model: {onnx_path}")
print(f"\n🎯 Detection Performance:")
print(f"  • Precision: {tp / (tp + fp):.4f}" if (tp + fp) > 0 else "  • Precision: N/A")
print(f"  • Recall: {tp / (tp + fn):.4f}" if (tp + fn) > 0 else "  • Recall: N/A")
print(f"  • F1-Score: {2 * (tp / (tp + fp)) * (tp / (tp + fn)) / ((tp / (tp + fp)) + (tp / (tp + fn))):.4f}" if (tp + fp) > 0 and (tp + fn) > 0 else "  • F1-Score: N/A")
print(f"\n✨ Ready for production deployment!")